In [12]:
2048 % 11

2

In [1]:
import numpy as np
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import datetime as dt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import joblib as jl

In [2]:
# Creating list of pathes to files
path = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data'

file_paths = []
i = 0
for root, dirs, files in os.walk(path, topdown = False):
    for name in files:
        if '3 sec' in name:
            file_paths.append(os.path.join(root, name))
            i+=1
print(i)

113


In [5]:
df = pd.read_csv(file_paths[0])
df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])

In [8]:
df.iloc[0,0].time() > dt.time(13,)

datetime.time(10, 58, 57, 12614)

In [26]:
df.iloc[3000:3060,:].mean(axis=1)

<ipython-input-26-25d2a3d657d3>:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df.iloc[3000:3060,:].mean(axis=1)


3000    50.007610
3001    51.989927
3002    53.603415
3003    55.216902
3004    56.825512
3005    58.426829
3006    60.152537
3007    61.513683
3008    62.874829
3009    62.010366
3010    61.145902
3011    60.269244
3012    61.116817
3013    61.964390
3014    62.807085
3015    63.494902
3016    64.076622
3017    65.452244
3018    66.827866
3019    68.143634
3020    69.758183
3021    69.459220
3022    70.818854
3023    72.067378
3024    71.593951
3025    70.924207
3026    70.679683
3027    70.468244
3028    69.711329
3029    69.807646
3030    69.771988
3031    69.885195
3032    68.159573
3033    66.541183
3034    65.131378
3035    64.220366
3036    62.971512
3037    61.088732
3038    59.131793
3039    56.895378
3040    54.980902
3041    52.546695
3042    50.170963
3043    48.176427
3044    47.616793
3045    47.160793
3046    45.892744
3047    44.941537
3048    44.769951
3049    44.708256
3050    44.427598
3051    44.145537
3052    44.117061
3053    43.953305
3054    43.644744
3055    43

In [152]:
train_list, test_list = [], []

for iter_num, file_path in enumerate(file_paths):
    
    # Cooking dataset for predicting
    df = pd.read_csv(file_path)

    # Filtering samples by time
    df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
    df = df[df.iloc[:, 0]>dt.]

    # Adding shifted time to check time correctness of sample
    df['Time_shift1'] = df.iloc[:, 0].shift(-20)
    df['Time_shift2'] = df.iloc[:, 0].shift(-40)
    df['Time_shift3'] = df.iloc[:, 0].shift(-60)

    # Adding future prices to be able to classify sample
    future_prices = pd.concat([df.iloc[:, 1:-3:41].shift(-20), 
                               df.iloc[:, 1:-3:41].shift(-40), 
                               df.iloc[:, 1:-3:41].shift(-60)], axis=1
                             )
    df['Max_3min'] = np.max(future_prices, axis=1)
    df['Min_3min'] = np.min(future_prices, axis=1)

    # Filter good samples
    df = df[(df.iloc[:, 821] - df.iloc[:, 0] < dt.timedelta(minutes=1, seconds=1)) &
            (df.iloc[:, 821] - df.iloc[:, 0] > dt.timedelta(minutes=0, seconds=59)) &
            (df.iloc[:, 822] - df.iloc[:, 0] < dt.timedelta(minutes=2, seconds=1)) &
            (df.iloc[:, 822] - df.iloc[:, 0] > dt.timedelta(minutes=1, seconds=59)) &
            (df.iloc[:, 823] - df.iloc[:, 0] < dt.timedelta(minutes=3, seconds=1)) &
            (df.iloc[:, 823] - df.iloc[:, 0] > dt.timedelta(minutes=2, seconds=59))
           ]
    df = df.dropna()

    # Filtering useful columns
    columns = [column for i, column in enumerate(df.columns) if not (('Time_shift' in column) | (i == 0))]
    df = df[columns]
    #df = df.astype('float32')

    # Renaming columns to group them
    new_columns = []
    j = 0
    group_size = 3
    for i, col in enumerate(columns):
        part_num = i//41
        if 'min' in col:
            new_columns.append(col)
        elif i%41 == 0:
            new_columns.append('price'+str(part_num))
        else:
            group_num = (i - part_num - 1) % 20 // group_size
            is_ask = (i - part_num - 1) // 20 % 2
            if is_ask:
                new_columns.append('ask'+str(group_num)+'_'+str(part_num))
            else: 
                new_columns.append('bid'+str(group_num)+'_'+str(part_num))

    # Grouping columns
    df.columns = new_columns
    df = df.groupby(df.columns, axis=1, sort=False).sum()

    # Group means
    price_cols = [col for col in df.columns if (('price' in col) | ('min' in col))]
    ob_cols = [col for col in df.columns if not (('price' in col) | ('min' in col))]
    renamed_ob_cols = [col[:4] for col in ob_cols]
    # df_test = df.copy()
    # df_test = df_test[ob_cols]
    # df_test.columns = renamed_ob_cols
    # print(df_test.groupby(df_test.columns, axis=1).mean().iloc[0])

    # Normalizing samples
    df[ob_cols] = np.divide(df[ob_cols].values, df[ob_cols].mean(axis=1).values.reshape(-1,1))
    df[price_cols] = np.divide(df[price_cols].values, df['price19'].values.reshape(-1,1)) - 1
    
    # Downsizing
    df = df.values.astype('float32')
    
    # Adding into data
    if '2021-06' in file_path:
        test_list.append(df)
    else:
        train_list.append(df)
    
    print('%i/%i'%(iter_num+1, len(file_paths)), end='\r')
    
train = np.vstack(train_list)
del train_list
test = np.vstack(test_list)
del test_list

In [153]:
jl.dump(train,'train.jl')
jl.dump(test,'test.jl')

['test.jl']

In [92]:
def add_features(df):
    # Price diffs
    df['price_diff1'] = df.iloc[:, :1].groupby(df.index).diff(1)
    df['price_diff2'] = df.iloc[:, :1].groupby(df.index).diff(2)
    df['price_diff3'] = df.iloc[:, :1].groupby(df.index).diff(3)
    df = df.fillna(0)
    
    # Amount of orders diff
    df['orders_diff1'] = df.iloc[:, 1:].mean(axis=1).groupby(df.index).diff(1)
    df['orders_diff2'] = df.iloc[:, 1:].mean(axis=1).groupby(df.index).diff(2)
    df['orders_diff3'] = df.iloc[:, 1:].mean(axis=1).groupby(df.index).diff(3)
    df = df.fillna(0)
    
    # Densities
    df['dense_diff1'] = np.where(df['orders_diff1'] == 0, 0, df['price_diff1'] / df['orders_diff1'])
    df['dense_diff2'] = np.where(df['orders_diff2'] == 0, 0, df['price_diff2'] / df['orders_diff2'])
    df['dense_diff3'] = np.where(df['orders_diff3'] == 0, 0, df['price_diff3'] / df['orders_diff3'])
    
    return df

In [127]:
data = df.iloc[:, :-2]
targets = df.iloc[:, -2:]
idxs = np.array(list(range(targets.shape[0]))*20).reshape(20, targets.shape[0]).T.reshape(-1)
data = pd.DataFrame(train.values.reshape(-1,15), index=idxs)
# targets = targets.values.reshape(-1,2)
data = data.values.reshape(-1,20,15)
targets = targets.values
data.shape, targets.shape

((11921, 20, 15), (11921, 2))

In [128]:
train_idx, val_idx = list(range(int(targets.shape[0]*0.8))), list(range(int(targets.shape[0]*0.8)+1, targets.shape[0]))
X_train, X_val = data[train_idx], data[val_idx]
y_train, y_val = targets[train_idx], targets[val_idx]

In [109]:
# LSTM
model = keras.Sequential()
model.add(keras.Input(shape=(None, 15)))
model.add(layers.LSTM(256, return_sequences=False, activation='tanh'))
model.add(layers.Dense(2))

In [111]:
model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mean_squared_error']
)

In [129]:
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10
149/149 - 14s - loss: 0.0053 - mean_squared_error: 0.0053 - 14s/epoch - 94ms/step
Epoch 2/10
149/149 - 11s - loss: 1.3207e-04 - mean_squared_error: 1.3207e-04 - 11s/epoch - 76ms/step
Epoch 3/10
149/149 - 11s - loss: 7.3130e-05 - mean_squared_error: 7.3130e-05 - 11s/epoch - 76ms/step
Epoch 4/10
149/149 - 12s - loss: 6.7332e-05 - mean_squared_error: 6.7332e-05 - 12s/epoch - 79ms/step
Epoch 5/10
149/149 - 11s - loss: 5.0169e-05 - mean_squared_error: 5.0169e-05 - 11s/epoch - 77ms/step
Epoch 6/10
149/149 - 13s - loss: 4.0506e-05 - mean_squared_error: 4.0506e-05 - 13s/epoch - 85ms/step
Epoch 7/10
149/149 - 12s - loss: 4.5434e-05 - mean_squared_error: 4.5434e-05 - 12s/epoch - 78ms/step
Epoch 8/10
149/149 - 12s - loss: 1.0812e-04 - mean_squared_error: 1.0812e-04 - 12s/epoch - 79ms/step
Epoch 9/10
149/149 - 12s - loss: 5.2261e-05 - mean_squared_error: 5.2261e-05 - 12s/epoch - 78ms/step
Epoch 10/10
149/149 - 12s - loss: 3.0032e-05 - mean_squared_error: 3.0032e-05 - 12s/epoch - 78ms/st

In [130]:
model.evaluate(X_val, y_val, batch_size=64, verbose=2)

38/38 - 2s - loss: 3.5492e-05 - mean_squared_error: 3.5492e-05 - 2s/epoch - 48ms/step


[3.549241955624893e-05, 3.549241955624893e-05]